In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from hyperopt import fmin, tpe, hp

2023-10-29 13:51:21.057816: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 13:51:21.088977: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-29 13:51:21.089016: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-29 13:51:21.089049: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-29 13:51:21.095258: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 13:51:21.096148: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
# Load the dataset
df = pd.read_csv("../data/data.csv") 

In [4]:
df

,Date,GemstoneName,Color,Clarity,Cut,Weight,Price
0,2021-01-15,Spinel,Brown,Transparent,Oval,1.36,9000
1,2021-01-15,Cat's Eye,Yellow,Translucent,Oval,0.75,18000
2,2021-01-17,Spinel,Brown,Transparent,Oval,1.36,9000
3,2021-01-28,Cat's Eye,Yellow,Translucent,Oval,0.75,18000
4,2021-02-12,Sapphire,Purple,Transparent,Emerald,0.73,20000
...,...,...,...,...,...,...,...
696,2022-05-11,Ruby,Red,Transparent,Emerald,0.51,30000
697,2022-08-04,Ruby,Red,Transparent,Emerald,0.95,30000
698,2022-09-20,Ruby,Purple,Transparent,Oval,0.73,30000
699,2023-01-17,Ruby,Purple,Transparent,Oval,0.73,30000


In [6]:
# Encode categorical features
categorical_columns = ['GemstoneName','Color','Clarity','Cut']
for column in categorical_columns:
    df[column] = LabelEncoder().fit_transform(df[column])


In [8]:
# Split the data into features and target
X = df.drop(columns=['Date', 'Price'])
y = df['Price']

In [9]:
# Normalize numerical features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Define a hyperparameter search space
space = {
    'n_hidden_layers': hp.choice('n_hidden_layers', [1, 2, 3]),
    'n_units': hp.choice('n_units', [64, 128, 256]),
    'learning_rate': hp.loguniform('learning_rate', -5, -2),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5),
    'epochs': hp.quniform('epochs', 50, 200, 10),
    'batch_size': hp.choice('batch_size', [32, 64, 128])
}

In [12]:
# Define the objective function for hyperparameter tuning
def objective(params):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(X_train.shape[1],)))

    for _ in range(params['n_hidden_layers']):
        model.add(keras.layers.Dense(params['n_units'], activation='relu'))
        model.add(keras.layers.Dropout(params['dropout_rate']))

    model.add(keras.layers.Dense(1))  # Output layer for regression

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=params['learning_rate']),
                  loss='mean_squared_error')

    model.fit(X_train, y_train, epochs=int(params['epochs']), batch_size=params['batch_size'], verbose=0)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse

In [13]:
# Perform hyperparameter tuning using Hyperopt
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100)

5/5 [==============================] - 0s 2ms/step    

5/5 [==============================] - 0s 2ms/step                              

5/5 [==============================] - 0s 2ms/step                             

5/5 [==============================] - 0s 2ms/step                             

5/5 [==============================] - 0s 2ms/step                             

5/5 [==============================] - 0s 1ms/step                             

5/5 [==============================] - 0s 2ms/step                             

5/5 [==============================] - 0s 1ms/step                             

5/5 [==============================] - 0s 2ms/step                             

5/5 [==============================] - 0s 2ms/step                             

5/5 [==============================] - 0s 1ms/step                              

5/5 [==============================] - 0s 2ms/step                              

5/5 [==============================] - 0s 1ms/step

In [14]:
# Train the final model with the best hyperparameters
final_model = keras.Sequential()
final_model.add(keras.layers.Input(shape=(X_train.shape[1],)))

In [15]:
for _ in range(best['n_hidden_layers']):
    final_model.add(keras.layers.Dense(best['n_units'], activation='relu'))
    final_model.add(keras.layers.Dropout(best['dropout_rate']))

final_model.add(keras.layers.Dense(1))

final_model.compile(optimizer=keras.optimizers.Adam(learning_rate=best['learning_rate']),
                    loss='mean_squared_error')

final_model.fit(X_train, y_train, epochs=int(best['epochs']), batch_size=best['batch_size'])

Epoch 1/180


18/18 [==============================] - 0s 1ms/step - loss: 9990151168.0000
Epoch 2/180
18/18 [==============================] - 0s 1ms/step - loss: 9989468160.0000
Epoch 3/180
18/18 [==============================] - 0s 1ms/step - loss: 9987553280.0000
Epoch 4/180
18/18 [==============================] - 0s 1ms/step - loss: 9983173632.0000
Epoch 5/180
18/18 [==============================] - 0s 1ms/step - loss: 9974564864.0000
Epoch 6/180
18/18 [==============================] - 0s 1ms/step - loss: 9958589440.0000
Epoch 7/180
18/18 [==============================] - 0s 1ms/step - loss: 9934145536.0000
Epoch 8/180
18/18 [==============================] - 0s 1ms/step - loss: 9899625472.0000
Epoch 9/180
18/18 [==============================] - 0s 1ms/step - loss: 9848049664.0000
Epoch 10/180
18/18 [==============================] - 0s 1ms/step - loss: 9781009408.0000
Epoch 11/180
18/18 [==============================] - 0s 1ms/step - loss: 9703561216.0000
Epoch 12/180
18/18 [===========

In [16]:
# Evaluate the final model
test_loss = final_model.evaluate(X_test, y_test)
print(f"Test RMSE: {np.sqrt(test_loss)}")

5/5 [==============================] - 0s 1ms/step - loss: 3811832064.0000


Test RMSE: 61740.03615159291


In [17]:
# Assuming you have trained and evaluated your final model as 'final_model'

# Save the model to a file
final_model.save('gemstone_price_prediction_model3.h5')


/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
# Predictions and RMSE Calculation
y_pred = final_model.predict(X_test).flatten()
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")

NameError: name 'model' is not defined